In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('nl_ranks.tsv', sep='\t')
display(df)

,word,count,cosine_distance,d_rank,e_rank,sum_rank
0,huh,51,0.159184,15,0,15
1,oké,88,0.202835,11,9,20
2,mam,37,0.202271,19,8,27
3,papa,41,0.218817,17,19,36
4,pap,30,0.211262,26,13,39
5,hè,21,0.181786,39,5,44
6,schatje,13,0.162617,58,1,59
7,hé,13,0.180575,63,4,67
8,lieverd,10,0.174831,93,2,95
9,eh,7,0.213423,136,15,151
